# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    amount_invested = []
    lis = []
    Beta = []
    Volatility = []
    for x in P["Amount Invested"]:
        amount_invested += [x]
    added = sum(amount_invested)
    for n in P["Beta"]:
        Beta += [n]
    for l in P["Volatility"]:
        Volatility += [l]
    for y in amount_invested:
        lis += [y / added]
    B_p = sum(list(map(lambda x,y: x*y,lis,Beta)))
    E = rf + (B_p*mkt_rp)
    return E
print(expected_return(portfolio, 0.02, 0.05))

0.086


In [4]:
def volatility(P, cor):
    amount_invested = P["Amount Invested"].tolist()
    Volatility = P["Volatility"].tolist()
    lis = []
    added = sum(amount_invested)
    n = len(P)
    var = 0 
    for y in amount_invested:
        lis += [y / added]
    for i in range(n):
        for j in range(i+1, n):
            var += 2 * (lis[i])*(Volatility[i])*(lis[j])*(Volatility[j]*cor)
    VAR_1 = sum(list(map(lambda x,y: (x**2)*(y**2),lis,Volatility)))
    VAR = VAR_1 + var
    SD = np.sqrt(VAR)
    return SD 
print(volatility(portfolio, -0.2))

0.24083189157584595


Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [5]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    x = vol/mkt_vol
    expectedreturn = rf + (x*mkt_rp)
    return expectedreturn
print(expected_return2(0.02, 0.05, 0.16, 0.2408))

0.09525


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [6]:
def CAPM_beta(return_path, factors_path):
    df = pd.read_csv(return_path)
    df_1 = pd.read_csv(factors_path)
    cusip = []
    exp_ret = []
    # adding month a date together
    df['month'] = df["month"].astype(str).str.zfill(2)
    df["date"] = df["year"].astype(str) + df["month"]
    df_1["date"] = df_1["date"].astype(str)
    df = df.drop(['year', 'month'], axis=1).set_index('date')
    df_1['Mkt-RF'] = df_1['Mkt-RF'].div(100)
    df_1['RF'] = df_1['RF'].div(100)
    out = pd.merge(df, df_1, how = 'left', on = ["date"])
    out['ri-rf'] = out['RET'].sub(out["RF"])
    out["R_mkt-Rf"] = out['Mkt-RF'].sub(out["RF"])    
    grouped = out.groupby('CUSIP')
    beta = []
    for col,row in grouped:
        cusip += [col]
        e_ret = sum(row["RET"])/len(row["RET"])
        exp_ret += [e_ret]
        ri_rf = list(map(lambda x: e_ret - x, row["RF"])) 
        mean_r = sum(ri_rf)/len(ri_rf)
        R_mkt_Rf = row["Mkt-RF"].tolist()
        mean_m = sum(R_mkt_Rf)/len(R_mkt_Rf)
        cov = (sum(list(map(lambda x,y: (x - mean_r) * (y - mean_m),ri_rf,R_mkt_Rf))))/(len(ri_rf)-1)
        var = (sum(list(map(lambda x: (x - mean_m)**2, R_mkt_Rf))))/(len(R_mkt_Rf)-1)
        beta += [cov/var] 
        
        
        
    output = pd.DataFrame(data = {
        "CUSIP" : cusip ,
        "EXP_RETURN" : exp_ret,
        "BETA" : beta,
    })
    return output
CAPM_beta("monthlycrsp_mod7.csv","F-F_Research_Data_Factors.CSV")

CUSIP  EXP_RETURN      BETA
0     00081T10    0.016252  0.002858
1     00108M10    0.011101  0.004684
2     00130H10    0.016152  0.001601
3     00163U10    0.015538  0.002305
4     00182C10    0.014772  0.002804
...        ...         ...       ...
3220   9959910    0.009933  0.005622
3221   9972410    0.012866  0.001858
3222   9972510    0.017177  0.006884
3223   9984910    0.009006 -0.001650
3224   9985510    0.015699  0.006091

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [7]:
def slope(data):
    x = data["BETA"].tolist()
    y = data["EXP_RETURN"].tolist()
    slope_ = np.polyfit(x,y,1)[0]
    # This should return 0.007981588120535642... with the provided data.
    return slope_
slope(CAPM_beta("monthlycrsp_mod7.csv","F-F_Research_Data_Factors.CSV"))

-0.08075006325632571